In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from __future__ import annotations

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

from network_utils import (
    create_56bus,
    create_RX_from_net,
    read_load_data)
from robust_voltage_control import VoltPlot

In [ ]:
def decentralized_voltage_control(
        p: np.ndarray, qe: np.ndarray,
        v_lims: tuple[Any, Any], q_lims: tuple[Any, Any], v_nom: Any,
        X: np.ndarray, R: np.ndarray, version: str,
        v_sub: float, eps_scale: float,
        pbar: tqdm | None = None,
        log: tqdm | io.TextIOBase | None = None,
        volt_plot: VoltPlot | None = None
        ) -> tuple[np.ndarray, np.ndarray]:
    """Runs robust voltage control.

    Args
    - p: np.array, shape [T, n], active power injection (MW)
    - qe: np.array, shape [T, n], exogenous reactive power injection (MVar)
    - v_lims: tuple (v_min, v_max), squared voltage magnitude limits (kV^2)
        - v_min, v_max could be floats, or np.arrays of shape [n]
    - q_lims: tuple (q_min, q_max), reactive power injection limits (MVar)
        - q_min, q_max could be floats, or np.arrays of shape [n]
    - v_nom: float or np.array of shape [n], desired nominal voltage
    - X: np.array, shape [n, n], line parameters for reactive power injection
    - R: np.array, shape [n, n], line parameters for active power injection
    - version: str, one of ['feasible', 'opt']
    - v_sub: float, fixed squared voltage magnitude at substation (kV^2)
    - eps_scale: float
    - pbar: optional tqdm, progress bar
    - volt_plot: VoltPlot

    Returns
    - vs: np.array, shape [T, n]
    - qcs: np.array, shape [T, n]
    """
    assert p.shape == qe.shape
    T, n = qe.shape

    if log is None:
        log = tqdm()

    v_min, v_max = v_lims
    q_min, q_max = q_lims

    v = np.zeros([T, n])  # v[t] denotes v(t)
    qc = np.zeros([T, n])  # qc[t] denotes q^c(t)

    vpars = qe @ X + p @ R + v_sub  # shape [T, n], vpars[t] denotes vpar(t)
    v[0] = vpars[0]

    lam_low = 0
    lam_high = 0

    sigmas = np.linalg.svd(X)[1]
    s_max = np.max(sigmas)
    s_min = np.min(sigmas)

    if version == 'feasible':
        eps = 2 * s_min / s_max**2
    elif version == 'opt':
        eps = 1. / s_max
    else:
        raise ValueError('unknown value for `version`')
    log.write(f'eps = {eps:.3g}')

    for t in range(T-1):  # t = 0, ..., T-2
        if version == 'feasible':
            d = np.maximum(0, v[t] - v_max) - np.maximum(0, v_min - v[t])
            qc[t+1] = qc[t] - eps * d
        elif version == 'opt':
            lam_high = np.maximum(0, lam_high + eps * (v[t] - v_max))
            lam_low = np.maximum(0, lam_low + eps * (v_min - v[t]))
            qc[t+1] = lam_low - lam_high

        qc[t+1] = np.maximum(q_min, qc[t+1])
        qc[t+1] = np.minimum(q_max, qc[t+1])

        v[t+1] = vpars[t+1] + qc[t+1] @ X
        if pbar is not None:
            pbar.update()

    # update voltplot at the end of run
    if volt_plot is not None:
        volt_plot.update(qcs=qc, vs=np.sqrt(v), vpars=np.sqrt(vpars),
                         dists=None)

    return v, qc

In [ ]:
net = create_56bus()
R, X = create_RX_from_net(net, noise=0)  # true R and X
p, qe = read_load_data()  # in MW and MVar
T, n = p.shape
print(T, n)

In [ ]:
# ==== FIXED PARAMETERS ====
v_min, v_max = (11.4**2, 12.6**2)  # +/-5%, units kV^2
v_nom = 12**2  # nominal squared voltage magnitude, units kV^2
v_sub = v_nom  # fixed squared voltage magnitude at substation, units kV^2

vpars = qe @ X + p @ R + v_sub  # shape [T, n]
Vpar_min = np.min(vpars, axis=0)  # shape [n]
Vpar_max = np.max(vpars, axis=0)  # shape [n]

Pv = 0.1
Pu = 10

# weights on slack variables: alpha for CBC, beta for robust oracle
alpha = 1000
beta = 100
# ==== end of FIXED PARAMETERS ====

q_max = 0.24

In [ ]:
volt_plot = VoltPlot(
    v_lims=(np.sqrt(v_min), np.sqrt(v_max)),
    q_lims=(-q_max, q_max))

vs, qcs = decentralized_voltage_control(
    p=p, qe=qe,
    v_lims=(v_min, v_max), q_lims=(-q_max, q_max), v_nom=v_nom,
    X=X, R=R, version='feasible',
    eps_scale=1, v_sub=v_sub,
    pbar=tqdm(), log=tqdm,
    volt_plot=volt_plot)

volt_plot.fig.savefig('out/decentralized_feasible.png', bbox_inches='tight')

In [ ]:
volt_plot = VoltPlot(
    v_lims=(np.sqrt(v_min), np.sqrt(v_max)),
    q_lims=(-q_max, q_max))

vs, qcs = decentralized_voltage_control(
    p=p, qe=qe,
    v_lims=(v_min, v_max), q_lims=(-q_max, q_max), v_nom=v_nom,
    X=X, R=R, version='opt',
    eps_scale=1, v_sub=v_sub,
    pbar=tqdm(), log=tqdm,
    volt_plot=volt_plot)

volt_plot.fig.savefig('out/decentralized_opt.png', bbox_inches='tight')